In [1]:
from attribution import GraphAttributions
from pathlib import Path
from train import init
import yaml
from captum.attr import visualization

Using backend: pytorch


In [2]:
config_file_path = Path("config.yaml")
with open(config_file_path, 'r') as f:
    config = yaml.safe_load(f)
setup = init(config['Setup'])

dataset_name = config['Setup']['dataset']
dataset_config = config['Datasets'][dataset_name]
mode = config['Setup']['mode']
lap_pos_enc = config['Setup']['lap_pos_enc']

model_config = config['Network']
model_config['mode'] = mode
model_config['lap_pos_enc'] = lap_pos_enc
model_config['num_vocab'] = dataset_config['num_vocab']
model_config['num_edge_type'] = dataset_config['num_edge_type']
model_config['device'] = setup['device']
model_config['n_class'] = dataset_config['n_class']  

Model dir already exists


In [3]:
graph_dir = Path('Dataset/Fake_News/v1/GT_graphs_10k/Graph')
graph_info_dir = Path('Dataset/Fake_News/v1/GT_graphs_10k/Graph_info')
model_path = Path('Experiments/FakeNews_init/model/model_epoch_4.pt')
val_list_path = Path("Experiments/FakeNews_init/train.txt")
output_dir = Path("Experiments/FakeNews_init/results")

In [4]:
with open(val_list_path, 'r') as f:
    val_list = f.read().split("\n")
attribution_driver = GraphAttributions(model_path, model_config, graph_dir, graph_info_dir)

In [5]:
records = []
for graph_id in val_list[20:30]:
    data = attribution_driver.get_attributions(graph_id)
    original_score, pred, attributions, texts, delta = data
    vi = visualization.VisualizationDataRecord(attributions, 0.1, str(pred),
                                               str(original_score), 0, attributions.sum(), texts, delta)
    records.append(vi)

/home/kayhoss/miniconda3/envs/pytorch/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: DGLGraph.adjacency_matrix_scipy is deprecated. Please replace it with:

	DGLGraph.adjacency_matrix(transpose, scipy_fmt="csr").

  return warnings.warn(message, category=category, stacklevel=1)


In [6]:
visualization.visualize_text(records)